In [32]:
# To download top 50 stocks

In [33]:
# libraries 

from getData import get_data, get_ohlcv
from technical_indicator import LR, MACD, RSI, BB, ATR, ADX, RENKO, anomaly, historical_volatility, get_support_resistance_levels, VWAP
from Returns import cummulative_returns, CAGR, volatility, SHARPE, SORTINO, MAXDROWDOWN, CALMAR
from signals import stratgey1_mean_reversion, label_sd_with_signals,label_exit_signals


from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import copy
import os

%load_ext autoreload
%autoreload 2
pd.set_option('mode.chained_assignment', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# tickers = ['ICICIBANK.NS']
# tickers

tickers = ['DOMS.NS', 'TRIVENI.NS', 'POLYMED.NS', 'UNITDSPR.NS', 
       'HCLTECH.NS', 'FLUOROCHEM.NS', 'LUPIN.NS', 'GICRE.NS']

In [41]:
# Store data in dictionary
ohlcv_dict = {}
for stock in tickers:
    ohlcv_dict[stock] = get_ohlcv(ticker=stock, period = '1y', interval='1d', multi_level_index=False)
    # ohlcv_dict[stock] = ohlcv_dict[stock].between_time('09:35', '16:00')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [42]:
data = copy.deepcopy(ohlcv_dict)

In [43]:
for stock_name in tickers:
    cummulative_returns(data[stock_name])
    ATR(data[stock_name])
    anomaly(data[stock_name])
    support, resistnace  = get_support_resistance_levels(data[stock_name])
    data[stock_name]['support'] = support
    data[stock_name]['resistance'] = resistnace
    historical_volatility(data[stock_name])
    historical_volatility(data[stock_name])
    RSI(data[stock_name])
    VWAP(data[stock_name])
    BB(data[stock_name])
    data[stock_name]['Close'] = round(data[stock_name]['Close'],2)
    data[stock_name]['Signal'] = data[stock_name].apply(stratgey1_mean_reversion, axis=1)
    data[stock_name]['m_half_50'] = data[stock_name]['Close'].ewm(halflife=50).mean()
    data[stock_name]['Distance_From_Mean'] = (data[stock_name]['Close'] - data[stock_name]['m_half_50'])
    data[stock_name]=label_sd_with_signals(data[stock_name])
    data[stock_name]=label_exit_signals(data[stock_name])
    print(f'Now processing {[stock_name]}')

Now processing ['DOMS.NS']
Now processing ['TRIVENI.NS']
Now processing ['POLYMED.NS']
Now processing ['UNITDSPR.NS']
Now processing ['ABFRL.NS']
Now processing ['HCLTECH.NS']
Now processing ['FLUOROCHEM.NS']
Now processing ['ABLBL.NS']
Now processing ['SAREGAMA.NS']
Now processing ['INDHOTEL.NS']
Now processing ['FSL.NS']
Now processing ['SARDAEN.NS']
Now processing ['PATANJALI.NS']
Now processing ['PGEL.NS']
Now processing ['CHAMBLFERT.NS']
Now processing ['LUPIN.NS']
Now processing ['GICRE.NS']


In [44]:
# data

In [45]:
for stock_name in tickers:
    df = data[stock_name]
    resistance = df.loc[df['resistance']>1]['resistance'].tail(5)
    resistance.name = 'Close'
    
    resistance_levels = [
        {"date": date.strftime("%Y-%m-%d"), "Close": round(Close,2)}
        for date, Close in resistance.items()
    ]
    # resistance_levels
    
    support = df.loc[df['support']>1]['support'].tail(5)
    support.name = 'Close'
    
    support_levels = [
        {"date": date.strftime("%Y-%m-%d"), "Close": round(Close,2)}
        for date, Close in support.items()
    ]
    
    

    fig = go.Figure()
    #Close
    fig.add_trace(go.Scatter(
        x=data[stock_name].index,
        y=data[stock_name]['Close'],
        mode='lines',  # Set the mode to 'lines' for a line chart
        name='Close'# Use the name for each line from the dictionary
    ))   

    fig.add_trace(go.Scatter(
        x=data[stock_name].index,
        y=data[stock_name]['m_half_50'],
        mode='lines',  # Set the mode to 'lines' for a line chart
        name='MB'# Use the name for each line from the dictionary
    ))
    
    
    for level in support_levels:
        fig.add_hline(
            y=level["Close"],
            line=dict(color="green", dash="dash"),
            annotation_text=f"Support ({level['Close']})",
            annotation_position="bottom left"
        )
    
    # Add resistance lines
    for level in resistance_levels:
        fig.add_hline(
            y=level["Close"],
            line=dict(color="red", dash="dash"),
            annotation_text=f"Resistance ({level['Close']})",
                    annotation_position="top left"
        )
    
    
    fig.add_trace(go.Scatter(
        x=data[stock_name][data[stock_name]['Label'] == 'BELOW 1 SD'].index,
        y=data[stock_name][data[stock_name]['Label'] == 'BELOW 1 SD']['Close'],
        mode='markers',
        name='BELOW 1 SD',
        marker=dict(color='green', size=6, symbol='circle')
    ))

    
    fig.add_trace(go.Scatter(
        x=data[stock_name][data[stock_name]['Label'] == 'ABOVE 2 SD'].index,
        y=data[stock_name][data[stock_name]['Label'] == 'ABOVE 2 SD']['Close'],
        mode='markers',
        name='ABOVE 2 SD',
        marker=dict(color='red', size=10, symbol='triangle-down')
    ))
    

    fig.add_trace(go.Scatter(
        x=data[stock_name][data[stock_name]['Label'] == 'ABOVE 3 SD'].index,
        y=data[stock_name][data[stock_name]['Label'] == 'ABOVE 3 SD']['Close'],
        mode='markers',
        name='Buy 3 SD',
        marker=dict(color='red', size=20, symbol='triangle-down')
    ))


    fig.add_trace(go.Scatter(
        x=data[stock_name][data[stock_name]['Label'] == 'BELOW 2 SD'].index,
        y=data[stock_name][data[stock_name]['Label'] == 'BELOW 2 SD']['Close'],
        mode='markers',
        name='BELOW 2 SD',
        marker=dict(color='green', size=10, symbol='triangle-up')
    ))



    fig.add_trace(go.Scatter(
        x=data[stock_name][data[stock_name]['Label'] == 'BELOW 3 SD'].index,
        y=data[stock_name][data[stock_name]['Label'] == 'BELOW 3 SD']['Close'],
        mode='markers',
        name='BELOW 3 SD',
        marker=dict(color='green', size=20, symbol='triangle-up')
    ))

    df = data[stock_name]
    exit_rows = df[df['Exit_Price'].notnull()]
    
    fig.add_trace(go.Scatter(
        x=exit_rows.index,
        y=exit_rows['Exit_Price'],
        mode='markers',
        name='Exit Price',
        marker=dict(color='blue', size=10, symbol='circle')
    ))

    
    # Customize the layout
    fig.update_layout(
        title=f"{stock_name} Signal Chart",
        xaxis_title="Year",
        yaxis_title="Cumulative Return",
        autosize=False,
        width=1100,
        height=800,
        template='plotly_white'
    )
        
    
    
    # Path where you want to save
    output_dir = "../PlotlyOutput"
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Full file path
    file_path = os.path.join(output_dir, f"{stock_name}.png")
    
    # Save the figure
    fig.write_image(file_path)

    print(f"Output saved for {stock_name}")

Output saved for DOMS.NS
Output saved for TRIVENI.NS
Output saved for POLYMED.NS
Output saved for UNITDSPR.NS
Output saved for ABFRL.NS
Output saved for HCLTECH.NS
Output saved for FLUOROCHEM.NS
Output saved for ABLBL.NS
Output saved for SAREGAMA.NS
Output saved for INDHOTEL.NS
Output saved for FSL.NS
Output saved for SARDAEN.NS
Output saved for PATANJALI.NS
Output saved for PGEL.NS
Output saved for CHAMBLFERT.NS
Output saved for LUPIN.NS
Output saved for GICRE.NS
